# Module 3 / Recherche du plus court chemin

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 

In [6]:
timetables = pd.read_csv(r'data_sncf/timetables.csv', on_bad_lines='skip', sep='\t' , engine='python')
agency = pd.read_csv(r'data_sncf/agency.csv')
calendar = pd.read_csv(r'data_sncf/calendar.csv')
calendar_dates = pd.read_csv(r'data_sncf/calendar_dates.csv')
routes = pd.read_csv(r'data_sncf/routes.csv')
stops = pd.read_csv(r'data_sncf/stops.csv')
stop_times = pd.read_csv(r'data_sncf/stop_times.csv')
datransfersta7 = pd.read_csv(r'data_sncf/transfers.csv')
trips = pd.read_csv(r'data_sncf/trips.csv')

Joincture des différents fichiers csv 

stops et stop_times sont lié par stop_id mais les valeurs dans les différentes ont des préfixes. Pour pouvoir les lié, nous recupérons les 8 derniers caracteres de chaque valeurs, qui correspondent bien dans les deux csv.

Aprés celà, nous allons rajouter une nouvelle colonne dans les deux Dataframe, qui nous permettra de bien faire joincture.

Procédé : Enregistrement des valeurs de stop_id (sans les préfix) dans une liste. Transformation de la liste en dataframe puis rajout d'une colonne dans nos csv avec les valeurs exploitables

# Devrons nous nous y prendre d'une autre maniere ?

In [4]:
li_stops=[]
for x in stops["stop_id"]:
    li_stops.append(x[-8:])

data_stops = pd.DataFrame(li_stops, columns=['_stop_id'])
stops['_stop_id'] = data_stops
stops

,stop_id,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,_stop_id
0,StopArea:OCE87381509,Gare de Mantes-la-Jolie,NaN,48.989687,1.703294,NaN,NaN,1,NaN,87381509
1,StopArea:OCE87415604,Gare de Vernon-Giverny,NaN,49.091286,1.478363,NaN,NaN,1,NaN,87415604
2,StopArea:OCE87415620,Gare de Gaillon-Aubevoye,NaN,49.174632,1.352518,NaN,NaN,1,NaN,87415620
3,StopArea:OCE87415877,Gare de Val-de-Reuil,NaN,49.275399,1.224609,NaN,NaN,1,NaN,87415877
4,StopArea:OCE87411207,Gare de Oissel,NaN,49.343042,1.101821,NaN,NaN,1,NaN,87411207
...,...,...,...,...,...,...,...,...,...,...
9171,StopPoint:OCETrain TER-87718411,Gare de Novillars,NaN,47.283906,6.132791,NaN,NaN,0,StopArea:OCE87718411,87718411
9172,StopPoint:OCETrain TER-87718429,Gare de Deluz,NaN,47.292454,6.199545,NaN,NaN,0,StopArea:OCE87718429,87718429
9173,StopPoint:OCETrain TER-87718437,Gare de Laissey,NaN,47.298881,6.233690,NaN,NaN,0,StopArea:OCE87718437,87718437
9174,StopPoint:OCENavette-87471003,Gare de Rennes,NaN,48.103515,-1.672327,NaN,NaN,0,StopArea:OCE87471003,87471003


In [48]:
li_stop_times=[]
for x in stops["stop_id"]:
    li_stop_times.append(x[-8:])

data_stop_times = pd.DataFrame(li_stop_times, columns=['_stop_id'])
stop_times['_stop_id'] = data_stop_times
#li_stop_times
stop_times.head(2000)


,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,_stop_id
0,OCESN037071R0100119847,23:05:00,23:05:00,StopPoint:OCECar TER-87381509,0,NaN,0,0,NaN,87381509
1,OCESN037071R0100119847,23:35:00,23:35:00,StopPoint:OCECar TER-87415604,1,NaN,0,0,NaN,87415604
2,OCESN037071R0100119847,23:55:00,23:55:00,StopPoint:OCECar TER-87415620,2,NaN,0,0,NaN,87415620
3,OCESN037071R0100119847,24:25:00,24:25:00,StopPoint:OCECar TER-87415877,3,NaN,0,0,NaN,87415877
4,OCESN037071R0100119847,24:45:00,24:45:00,StopPoint:OCECar TER-87411207,4,NaN,0,0,NaN,87411207
...,...,...,...,...,...,...,...,...,...,...
1995,OCESN003355F190197816,20:34:00,20:36:00,StopPoint:OCETrain TER-87387001,1,NaN,0,0,NaN,87696658
1996,OCESN003355F190197816,21:02:00,21:03:00,StopPoint:OCETrain TER-87444299,2,NaN,0,0,NaN,87741702
1997,OCESN003355F190197816,21:19:00,21:20:00,StopPoint:OCETrain TER-87444265,3,NaN,0,0,NaN,87741710
1998,OCESN003355F190197816,21:48:00,21:48:00,StopPoint:OCETrain TER-87444000,4,NaN,0,0,NaN,87696443


In [ ]:
x = pd.merge(timetables, trips, on = "trip_id")

In [ ]:
y = pd.merge(x, stop_times, on = "trip_id")